### Proyecto Datos Masivos
- Ortega Ibarra Jaime Jesus.
1. Genera un buscador de documentos usando índice inverso e identifica a los usuarios con
gustos similares usando similitud de Jaccard, MinMax y Coseno.

Datos por utilizar:\
https://www.kaggle.com/CooperUnion/anime-recommendations-database.

Instalamos recursos necesarios para poder utilizar Spark dentro de nuestro entorno

In [0]:
#!apt-get install openjdk-8-jdk-headless -qq > /dev/null
#!wget -q http://www-eu.apache.org/dist/spark/spark-2.4.4/spark-2.4.4-bin-hadoop2.7.tgz
#!tar xf spark-2.4.4-bin-hadoop2.7.tgz
#!pip install -q findspark

In [41]:
pip install pyspark

In [42]:
pip install spark

In [0]:
from google.colab import files
import pandas as pd
import numpy as np
import io
from pyspark import SparkContext
import pyspark
import spark
from collections import Counter
from pyspark import SparkConf as scf
from sklearn.feature_extraction.text import CountVectorizer

In [44]:
uploaded = files.upload()

Saving rating.csv to rating (2).csv


In [0]:
conf = scf().setMaster('local').setAppName('ParallelizedCollections')
sc = SparkContext(conf = conf)

In [0]:
from pyspark.sql import SparkSession
spark = SparkSession.builder \
     .master("local") \
     .appName("Word Count") \
     .config("spark.some.config.option", "some-value") \
     .getOrCreate()

In [48]:
file_location = "rating.csv"
file_type = "csv"

infer_schema = "true"
first_row_is_header = "true"
delimiter = ","

df = spark.read.format(file_type) \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .load(file_location)

df.show()

+-------+--------+------+
|user_id|anime_id|rating|
+-------+--------+------+
|      1|      20|    -1|
|      1|      24|    -1|
|      1|      79|    -1|
|      1|     226|    -1|
|      1|     241|    -1|
|      1|     355|    -1|
|      1|     356|    -1|
|      1|     442|    -1|
|      1|     487|    -1|
|      1|     846|    -1|
|      1|     936|    -1|
|      1|    1546|    -1|
|      1|    1692|    -1|
|      1|    1836|    -1|
|      1|    2001|    -1|
|      1|    2025|    -1|
|      1|    2144|    -1|
|      1|    2787|    -1|
|      1|    2993|    -1|
|      1|    3455|    -1|
+-------+--------+------+
only showing top 20 rows



In [0]:
temp_table_name = "rating_csv"

df.createOrReplaceTempView(temp_table_name)

In [0]:
data_file = "rating.csv"
raw_rdd = sc.textFile(data_file)

In [0]:
data_file = "anime.csv"
raw_rdd = sc.textFile(data_file).cache()
raw_rdd

anime.csv MapPartitionsRDD[19] at textFile at NativeMethodAccessorImpl.java:0

In [58]:
csv_rdd = raw_rdd.map(lambda row: row.split(","))
print(csv_rdd.take(5))

[['user_id', 'anime_id', 'rating'], ['1', '20', '-1'], ['1', '24', '-1'], ['1', '79', '-1'], ['1', '226', '-1']]


In [59]:
from pyspark.sql import Row

anime_rdd = csv_rdd.map(lambda r: Row(
  user_id=r[0],
  anime_id=r[1],
  rating=r[2]
  )
)
anime_rdd.take(10)

[Row(anime_id='anime_id', rating='rating', user_id='user_id'),
 Row(anime_id='20', rating='-1', user_id='1'),
 Row(anime_id='24', rating='-1', user_id='1'),
 Row(anime_id='79', rating='-1', user_id='1'),
 Row(anime_id='226', rating='-1', user_id='1'),
 Row(anime_id='241', rating='-1', user_id='1'),
 Row(anime_id='355', rating='-1', user_id='1'),
 Row(anime_id='356', rating='-1', user_id='1'),
 Row(anime_id='442', rating='-1', user_id='1'),
 Row(anime_id='487', rating='-1', user_id='1')]

In [60]:
reviews_rdd = df.rdd.flatMap(lambda x: x)
reviews_rdd.take(10)

[1, 20, -1, 1, 24, -1, 1, 79, -1, 1]

In [7]:
data = pd.read_csv('rating.csv')
data.head(10)

,user_id,anime_id,rating
0,1,20,-1
1,1,24,-1
2,1,79,-1
3,1,226,-1
4,1,241,-1
5,1,355,-1
6,1,356,-1
7,1,442,-1
8,1,487,-1
9,1,846,-1


In [0]:
recorte = len(data)//50
data = data.iloc[:recorte,:]
data.to_csv("rating_split.csv", index=False)
reg = open("rating_split.csv", "r").readlines()[1:]
lista_datos=[]
for datos in reg:
    dat = datos.replace('\n','')
    lista_datos.append(dat)
#lista_datos

Como observamos, recortamos nuestros datos, pues al realizar la búsqueda con los datos completos era demasiado tiempo el que se necesitaba para su ejecución.

Creamos nuestras bolsas

In [0]:
v = CountVectorizer(lista_datos)
bolsas = v.fit_transform(lista_datos)

## Creamos la función Indice Inverso
Me he basado en la función que el Doctor ha proporcionado en clase.

In [0]:
class IndiceInverso:
    def  __getitem__(self, idx):
        return self.ifs[idx]

    def recupera(self, l):
        docs = Counter()
        for (i,_) in l:
            docs.update(self.ifs[i])

        return docs

    def from_csr(self, csr):
        self.ifs = [[] for _ in range(csr.shape[1])]
        coo = csr.tocoo()    
        for i,j,v in zip(coo.row, coo.col, coo.data):
            self.ifs[j].append(i)

In [0]:
ifs = IndiceInverso()
ifs.from_csr(bolsas)

In [0]:
def csr_to_ldb(csr):
    ldb = [[] for _ in range(csr.shape[0])]
    coo = csr.tocoo()    
    for i,j,v in zip(coo.row, coo.col, coo.data):
        ldb[i].append((j, v))

    return ldb

Realizamos consultas.

En este caso tome 3 de los primeros valores dentro de nuestro dataframe y los añadí a una lista vacía.

In [13]:
consultas =[]
for i in lista_datos[2:5]:
  print(i)
  consultas.append(i)
bc = v.transform(consultas)
cl = csr_to_ldb(bc)

1,79,-1
1,226,-1
1,241,-1


Recuperamos de acuerdo a la lista creada anterior mente con las consultas.

In [14]:
for i in range(len(cl)):
  recs = ifs.recupera(cl[i])
  top = recs.most_common()[i]
  print(lista_datos[top[0]])

1,79,-1
4,226,-1
54,241,-1


# Buscando Usuarios Similares.

Creamos las librerías para las distancias con ayuda del código visto en clase, posteriormente realizamos la funcion distancia que calculará las distintas distancias de acuerdo a lo que el usuario solicite y la función buscar, la cual realizará la búsqueda entre los datos.

In [0]:
def fuerza_bruta(base, consulta, fd):
    medidas = np.zeros(base.shape[0])
    for i,x in enumerate(base):
        medidas[i] = fd(consulta, x)
    return medidas

def similitud_jaccard(x, y):
  x = x.toarray()[0]
  y = y.toarray()[0]
  inter = np.count_nonzero(x * y)
  return inter / (np.count_nonzero(x) + np.count_nonzero(y) - inter)

def similitud_minmax(x, y):
  x = x.toarray()[0]
  y = y.toarray()[0]
  c = np.vstack((x,y))
  mn = np.sum(np.min(c, axis=0))
  mx = np.sum(np.max(c, axis=0))
  return mn / mx

def similitud_coseno(x, y):
  x = x.toarray()[0]
  y = y.toarray()[0]
  return (x @ y) / (np.sqrt(x @ x) * np.sqrt(y @ y))

def distancia(query, dist):
  #De acuerdo con las fórmulas vistas en clase, podemos calcular cada una de las distancias
  #de la siguiente manerá, el cálculo dependerá de la distancia que el usuario indique.
  bolsa_busqueda = v.transform([query])
  if dist == 'coseno':
    sim = fuerza_bruta(bolsas[1:],bolsa_busqueda, similitud_coseno)
  if dist == 'jaccard':
    sim = fuerza_bruta(bolsas[1:], bolsa_busqueda, similitud_jaccard)
  if dist == 'minmax':
    sim = fuerza_bruta(bolsas[1:], bolsa_busqueda, similitud_minmax)
  return sim
  
def busqueda(query, cant, dist,data):
  user = set()
  longitud = len(user)
  lista=[]
  x = 1
  sim = distancia(query, dist)
  while x <= len(lista_datos) and (longitud < cant):
    indices = np.nanargmax(sim)+1
    us = data.loc[indices,"user_id"]
    if us not in user:
      user.add(us)
      lista.append(indices)
      sim[indices-1]=0
    x = x+1
  return data.iloc[lista,:]

## Probamos con las distintas distancias

In [18]:
#Distancias
distancias = ['coseno', 'jaccard', 'minmax']
for i in distancias:
  print('\n',i)
  print(busqueda('3 ,527',5,i,data))


 coseno


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:24: RuntimeWarning: invalid value encountered in true_divide


        user_id  anime_id  rating
163           3       527       7
365           5       527       6
1328         13       527      -1
2824         29       527       8
3088         32       527      -1
...         ...       ...     ...
147328     1492       527      10
148791     1500       527      10
154579     1537       527      10
155437     1549       527      10
1             1        24      -1

[198 rows x 3 columns]

 jaccard
        user_id  anime_id  rating
163           3       527       7
365           5       527       6
1328         13       527      -1
2824         29       527       8
3088         32       527      -1
...         ...       ...     ...
147328     1492       527      10
148791     1500       527      10
154579     1537       527      10
155437     1549       527      10
1             1        24      -1

[198 rows x 3 columns]

 minmax
        user_id  anime_id  rating
163           3       527       7
365           5       527       6
1328         13

Este código ya lo había realizado, pero el día de ayer mientras terminaba de maquillar mi código para enviarlo me di cuenta que debía utilizar spark, comencé a realizarlo tanto en databricks como en colab, pero me confundí bastante con el manejo de spark y no pude concluirlo.

In [210]:
data = distancia('20','coseno')
type(data)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:24: RuntimeWarning: invalid value encountered in true_divide


numpy.ndarray

In [214]:
distancia('1 ,20','coseno')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:24: RuntimeWarning: invalid value encountered in true_divide


<class 'numpy.ndarray'>
